In [1]:
import numpy as np
import pandas as pd
import os

## Funções e Configurações

In [2]:
# this dictionary contains the configuration necessary to divide each dataset used in the work, the script is made
# for the conll type of datasets, so if you want to add datasets to divide, make sure to add it here
gen_config = {
    'leNER':{
        'base_dir': "../Base de Dados/leNER/",
        'load_dir': "documents/allDoc/",
        'save_dir': "divisions/iterative/",
        'col_sep' : ' ',
        'file_type': '.conll',
    },
    'UlyssesNER-BR':{
        'base_dir': "../Base de Dados/UlyssesNER-BR/",
        'load_dir': "PL_Corpus_byTypes_conll/",
        'save_dir': "divisions/iterative/",
        'col_sep' : ' ',
        'file_type': '.conll',
    },
    'Harem-second':{
        'base_dir': "../Base de Dados/HAREM/",
        'load_dir': "SecondHarem/documents",
        'save_dir': "divisions/second/iterative/",
        'col_sep' : '\t',
        'file_type': '.conll',
    },
    'Harem-first':{
        'base_dir': "../Base de Dados/HAREM/",
        'load_dir': "fistHarem/documents",
        'save_dir': "divisions/first/iterative/",
        'col_sep' : '\t',
        'file_type': '.conll',
    }
}

In [3]:
def get_sentencas(corpus, coll_sep):
        sentenca = []
        for line in corpus:
            if line == '\n':
                if sentenca:
                    yield sentenca
                    sentenca = []
            else:
                
                sentenca.append(line.strip('\n').split(coll_sep))
        if sentenca:
            yield sentenca

In [4]:
def read_dataset(filepath,coll_sep):
    data_por_sentenca = []
    with open(filepath, "r") as corpus:
        data_por_sentenca += list(get_sentencas(corpus,coll_sep))
    
    return data_por_sentenca

In [5]:
def get_data(config, selective = False, fileName = ""):
    load_dir = config['base_dir'] + config['load_dir']
    sep = config['col_sep']
    if selective:
        fp = load_dir + fileName
        return read_dataset(filepath=fp, coll_sep=sep)
    else:
        filepaths = []
        for file in os.listdir(load_dir):
            if file.endswith(config['file_type']):
                filepaths.append(load_dir + file)
        
        data_por_sentenca = []
        for filepath in filepaths:
            print(filepath+"\n")
            data_por_sentenca += read_dataset(filepath=filepath, coll_sep=sep)
        
        return data_por_sentenca

In [6]:
def split_token_label(data):
    return list(map(list,zip(*data)))

## Carregando DataSet e preparando dados

In [7]:
# dataset = 'leNER'
# dataset = 'Paramopama'
# dataset = 'Harem-mini'
dataset = 'Harem-first'
# dataset = 'UlyssesNER-BR'

config = gen_config[dataset]

filepaths = []

try:
    # Listar arquivos na pasta especificada no config
    full_path = config['base_dir'] + config['load_dir']
    files = os.listdir(full_path)
    print(f"Arquivos encontrados em {full_path}: {files}")
    for file in files:
        # Adicionar o caminho completo de cada arquivo à lista
        filepaths.append(full_path + file)
except FileNotFoundError:
    print(f"Erro: O diretório {full_path} não foi encontrado. Verifique o caminho no 'config'.")

# Caminho específico para o diretório de documentos do HAREM
folder_path = '../Base de Dados/HAREM/firstHarem/documents'
data_por_sentenca_por_doc = {}

# Verificar se o diretório existe antes de tentar acessá-lo
if os.path.exists(folder_path):
    for file in os.listdir(folder_path):
        filepath = os.path.join(folder_path, file)
        
        # Verificar se o item listado é um arquivo
        if os.path.isfile(filepath):
            try:
                # Ler o dataset e armazenar no dicionário
                data_por_sentenca_por_doc[file] = read_dataset(filepath, coll_sep=config['col_sep'])
                print(f"Documento carregado: {file}")  # Exibir o nome do documento carregado
            except Exception as e:
                print(f"Erro ao processar o arquivo {file}: {e}")
else:
    print(f"O diretório {folder_path} não existe. Verifique o caminho e tente novamente.")

# Exibir o total de documentos carregados e seus nomes
total_documentos = len(data_por_sentenca_por_doc)
print(f"Total de documentos carregados: {total_documentos}")

if total_documentos > 0:
    print("Nomes dos documentos carregados:")
    for doc in data_por_sentenca_por_doc.keys():
        print(doc)
else:
    print("Nenhum documento foi carregado.")


Erro: O diretório ../Base de Dados/HAREM/fistHarem/documents não foi encontrado. Verifique o caminho no 'config'.
Documento carregado: HAREM-594-08181
Documento carregado: HAREM-381-04080
Documento carregado: HAREM-859-03643
Documento carregado: HAREM-081-09224
Documento carregado: HAREM-866-01320
Documento carregado: HAREM-868-02153
Documento carregado: HAREM-765-05370
Documento carregado: HAREM-27C-00833
Documento carregado: HAREM-88H-09142
Documento carregado: HAREM-962-09734
Documento carregado: HAREM-498-05317
Documento carregado: HAREM-56H-02765
Documento carregado: HAREM-58H-03548
Documento carregado: HAREM-182-08895
Documento carregado: HAREM-49B-05700
Documento carregado: HAREM-86B-06279
Documento carregado: HAREM-367-04886
Documento carregado: HAREM-373-05583
Documento carregado: HAREM-065-06662
Documento carregado: HAREM-091-04065
Documento carregado: HAREM-464-07791
Documento carregado: HAREM-69G-06024
Documento carregado: HAREM-955-05032
Documento carregado: HAREM-591-0066

In [8]:
# data_por_sentenca_por_doc['HAREMdoc_aa64686'][0]
# len(data_por_sentenca_por_doc['HAREMdoc_aa64686'])
# data_por_sentenca_por_doc[list(data_por_sentenca_por_doc.keys())[0]]
# d = list(data_por_sentenca_por_doc.values())[0][10][-1]
# list(map(list,zip(d)))
# # d

In [9]:
from collections import Counter
labels_por_doc = {}

for docName, docData in data_por_sentenca_por_doc.items():
    allDocData = []
    
    for sentence in docData:
        allDocData += [i for i in sentence]
    
    docTokens, docLabels = split_token_label(allDocData)
    
    
    labels_por_doc[docName] = docLabels

    labels_count = Counter(docLabels)
    
    # Armazenar a contagem de rótulos para o documento atual
    labels_por_doc[docName] = dict(labels_count)

# Exibir as contagens de rótulos por documento
for doc, labels_count in labels_por_doc.items():
    print(f"Documento: {doc}")
    print("Contagem de rótulos:", labels_count)
    print()

Documento: HAREM-594-08181
Contagem de rótulos: {'O': 113, 'B-ORGANIZACAO': 1, 'B-LOCAL': 4, 'B-TEMPO': 2, 'B-PESSOA': 6, 'I-PESSOA': 4, 'I-LOCAL': 3, 'I-TEMPO': 1}

Documento: HAREM-381-04080
Contagem de rótulos: {'B-ORGANIZACAO': 6, 'I-ORGANIZACAO': 8, 'O': 260, 'B-TEMPO': 5, 'B-LOCAL': 5, 'B-PESSOA': 4, 'I-PESSOA': 5, 'B-COISA': 4, 'B-ABSTRACCAO': 4, 'B-VALOR': 4, 'I-VALOR': 3, 'I-ABSTRACCAO': 2}

Documento: HAREM-859-03643
Contagem de rótulos: {'O': 699, 'B-PESSOA': 41}

Documento: HAREM-081-09224
Contagem de rótulos: {'B-ORGANIZACAO': 9, 'O': 270, 'I-ORGANIZACAO': 18, 'B-LOCAL': 8, 'B-PESSOA': 1, 'I-PESSOA': 2, 'B-VALOR': 8, 'B-ABSTRACCAO': 5, 'I-ABSTRACCAO': 1, 'B-ACONTECIMENTO': 5, 'I-ACONTECIMENTO': 11, 'B-TEMPO': 13, 'I-LOCAL': 1, 'B-COISA': 1}

Documento: HAREM-866-01320
Contagem de rótulos: {'O': 328, 'B-ABSTRACCAO': 3, 'I-ABSTRACCAO': 6, 'B-COISA': 2}

Documento: HAREM-868-02153
Contagem de rótulos: {'B-ABSTRACCAO': 8, 'O': 206, 'B-OBRA': 2, 'I-OBRA': 8, 'B-VALOR': 2, 'B-OR

In [10]:
# labels_por_doc['HAREMdoc_aa64686'][-12]
# len(labels_por_doc['HAREMdoc_aa64686'])

In [11]:
from collections import Counter

# Inicializar um dicionário para armazenar contagens de rótulos por documento
# antes e depois de simplificar.
labels_por_doc = {}
labels_por_doc_simplificado = {}

for docName, docData in data_por_sentenca_por_doc.items():
    allDocData = []
    
    # Concatenar todas as sentenças de um documento em uma lista
    for sentence in docData:
        allDocData += [i for i in sentence]
    
    # Separar tokens e rótulos
    docTokens, docLabels = split_token_label(allDocData)
    
    # Contagem de rótulos com os prefixos (original)
    labels_count_original = Counter(docLabels)
    labels_por_doc[docName] = dict(labels_count_original)
    
    # Extração da última parte do rótulo (ex. 'B-PER' vira 'PER')
    labels_simplificados = [lbl.split('-')[-1] for lbl in docLabels]
    labels_count_simplificado = Counter(labels_simplificados)
    labels_por_doc_simplificado[docName] = dict(labels_count_simplificado)

# Exibir a contagem original e simplificada para comparação
for doc in labels_por_doc.keys():
    print(f"Documento: {doc}")
    print("Contagem de rótulos (original):", labels_por_doc[doc])
    print("Contagem de rótulos (simplificado):", labels_por_doc_simplificado[doc])
    print()

# Exibir o total de documentos
print("Total de documentos:", len(labels_por_doc))



Documento: HAREM-594-08181
Contagem de rótulos (original): {'O': 113, 'B-ORGANIZACAO': 1, 'B-LOCAL': 4, 'B-TEMPO': 2, 'B-PESSOA': 6, 'I-PESSOA': 4, 'I-LOCAL': 3, 'I-TEMPO': 1}
Contagem de rótulos (simplificado): {'O': 113, 'ORGANIZACAO': 1, 'LOCAL': 7, 'TEMPO': 3, 'PESSOA': 10}

Documento: HAREM-381-04080
{'B-ORGANIZACAO': 6, 'I-ORGANIZACAO': 8, 'O': 260, 'B-TEMPO': 5, 'B-LOCAL': 5, 'B-PESSOA': 4, 'I-PESSOA': 5, 'B-COISA': 4, 'B-ABSTRACCAO': 4, 'B-VALOR': 4, 'I-VALOR': 3, 'I-ABSTRACCAO': 2}
Contagem de rótulos (simplificado): {'ORGANIZACAO': 14, 'O': 260, 'TEMPO': 5, 'LOCAL': 5, 'PESSOA': 9, 'COISA': 4, 'ABSTRACCAO': 6, 'VALOR': 7}

Documento: HAREM-859-03643
Contagem de rótulos (original): {'O': 699, 'B-PESSOA': 41}
Contagem de rótulos (simplificado): {'O': 699, 'PESSOA': 41}

Documento: HAREM-081-09224
Contagem de rótulos (original): {'B-ORGANIZACAO': 9, 'O': 270, 'I-ORGANIZACAO': 18, 'B-LOCAL': 8, 'B-PESSOA': 1, 'I-PESSOA': 2, 'B-VALOR': 8, 'B-ABSTRACCAO': 5, 'I-ABSTRACCAO': 1, 'B-A

In [12]:
# labels_por_doc['HAREMdoc_aa64686'][-12]
# len(labels_por_doc['HAREMdoc_aa64686'])

In [13]:
lbl_list = []
for lbl in labels_por_doc.values():
    lbl_list.extend(lbl)  # Adiciona os rótulos de cada documento à lista

# Filtrando 'B-OUTRO' e 'I-OUTRO'
lbl_list = [label for label in lbl_list if label not in []]

# Obtendo os rótulos únicos
unique_lbl_list = list(set(lbl_list))

# Exibindo a lista de rótulos únicos
print(unique_lbl_list)




['I-PESSOA', 'B-LOCAL', 'B-VALOR', 'B-OBRA', 'I-VALOR', 'B-OUTRO', 'I-COISA', 'B-PESSOA', 'I-TEMPO', 'I-LOCAL', 'B-ABSTRACCAO', 'B-ORGANIZACAO', 'I-OBRA', 'B-COISA', 'B-ACONTECIMENTO', 'O', 'I-ABSTRACCAO', 'B-TEMPO', 'I-OUTRO', 'I-ACONTECIMENTO', 'I-ORGANIZACAO']


In [14]:
unique_lbl_list.pop(unique_lbl_list.index('O'))
unique_lbl_list

['I-PESSOA',
 'B-LOCAL',
 'B-VALOR',
 'B-OBRA',
 'I-VALOR',
 'B-OUTRO',
 'I-COISA',
 'B-PESSOA',
 'I-TEMPO',
 'I-LOCAL',
 'B-ABSTRACCAO',
 'B-ORGANIZACAO',
 'I-OBRA',
 'B-COISA',
 'B-ACONTECIMENTO',
 'I-ABSTRACCAO',
 'B-TEMPO',
 'I-OUTRO',
 'I-ACONTECIMENTO',
 'I-ORGANIZACAO']

In [15]:
unique_lbl_list = sorted(unique_lbl_list)
unique_lbl_list

['B-ABSTRACCAO',
 'B-ACONTECIMENTO',
 'B-COISA',
 'B-LOCAL',
 'B-OBRA',
 'B-ORGANIZACAO',
 'B-OUTRO',
 'B-PESSOA',
 'B-TEMPO',
 'B-VALOR',
 'I-ABSTRACCAO',
 'I-ACONTECIMENTO',
 'I-COISA',
 'I-LOCAL',
 'I-OBRA',
 'I-ORGANIZACAO',
 'I-OUTRO',
 'I-PESSOA',
 'I-TEMPO',
 'I-VALOR']

In [16]:
#test_l = ['LOCAL', 'ORGANIZACAO', 'PESSOA', 'TEMPO', 'LOCAL', 'ORGANIZACAO', 'PESSOA', 'TEMPO', 'LOCAL', 'ORGANIZACAO', 'PESSOA', 'TEMPO', 'TEMPO', 'TEMPO']
#[test_l.count(x) for x in unique_lbl_list]

In [17]:
from collections import Counter

initial_set = []
for s_idx, s_labels in enumerate(list(labels_por_doc.values())):
    # Contagem eficiente dos rótulos usando Counter
    label_count = Counter(s_labels)
    
    # Adicionando o índice do documento e as contagens para cada rótulo em unique_lbl_list
    initial_set.append([s_idx] + [label_count.get(x, 0) for x in unique_lbl_list])

# Verificando o tamanho de initial_set (número de documentos)
print(len(initial_set))


129


In [18]:
# initial_set[:, 1:]

In [19]:
# Adicionando a linha final com os totais de cada label (soma dos rótulos)
initial_set.append(['-'] + [sum(x) for x in zip(*(row[1:] for row in initial_set))])

# Agora vamos filtrar a matriz para manter apenas as 7 primeiras colunas
# (assumindo que as 7 primeiras colunas correspondem aos rótulos -B)
filtered_initial_set = []

for row in initial_set:
    # Mantendo apenas as 7 primeiras colunas (a primeira coluna é '-')
    filtered_row = row[:11]  # Considerando que as primeiras 7 colunas são os rótulos -B
    filtered_initial_set.append(filtered_row)

# Exibindo a matriz resultante filtrada
for row in filtered_initial_set:
    print(row)


[0, 0, 0, 0, 4, 0, 1, 0, 6, 2, 0]
[1, 4, 0, 4, 5, 0, 6, 0, 4, 5, 4]
[2, 0, 0, 0, 0, 0, 0, 0, 41, 0, 0]
[3, 5, 5, 1, 8, 0, 9, 0, 1, 13, 8]
[4, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[5, 8, 0, 0, 2, 2, 5, 1, 1, 4, 2]
[6, 4, 0, 0, 13, 0, 2, 0, 0, 9, 10]
[7, 1, 3, 2, 40, 10, 55, 0, 73, 8, 14]
[8, 0, 0, 0, 3, 0, 1, 0, 0, 2, 1]
[9, 1, 0, 0, 3, 7, 9, 0, 4, 0, 3]
[10, 0, 1, 1, 4, 0, 0, 0, 0, 0, 1]
[11, 2, 0, 0, 0, 0, 8, 0, 0, 1, 1]
[12, 0, 0, 0, 4, 0, 8, 0, 0, 1, 2]
[13, 0, 0, 0, 5, 5, 9, 0, 15, 1, 4]
[14, 6, 0, 0, 27, 3, 1, 0, 3, 2, 3]
[15, 7, 11, 1, 19, 4, 28, 0, 15, 20, 5]
[16, 17, 2, 1, 3, 0, 7, 0, 7, 3, 0]
[17, 0, 2, 0, 1, 4, 0, 1, 4, 2, 3]
[18, 7, 1, 0, 9, 1, 5, 0, 1, 1, 0]
[19, 6, 6, 0, 15, 0, 9, 0, 2, 6, 1]
[20, 0, 0, 0, 3, 0, 2, 0, 2, 0, 5]
[21, 3, 0, 0, 26, 4, 62, 0, 34, 2, 2]
[22, 2, 0, 0, 27, 0, 2, 0, 0, 0, 2]
[23, 1, 1, 0, 5, 1, 17, 0, 1, 9, 2]
[24, 0, 1, 4, 3, 0, 1, 0, 1, 3, 2]
[25, 5, 0, 0, 1, 8, 12, 0, 4, 4, 4]
[26, 17, 3, 2, 3, 0, 9, 3, 0, 3, 2]
[27, 0, 1, 0, 3, 3, 1, 0, 4, 5, 2]
[28, 

In [20]:
# list(labels_por_doc.keys())

In [21]:
#initial_set[-1] = initial_set[-1][:7]  # Isso mantém as 7 primeiras colunas, incluindo '-'

# Exibindo a última linha após a atualização
print(filtered_initial_set[-1])


['-', 408, 129, 138, 1237, 197, 930, 42, 1074, 441, 468]


In [22]:
len(filtered_initial_set)

130

In [23]:
number_divisions = 10
result = [x / number_divisions for x in filtered_initial_set[-1][1:]]
print(result)


[40.8, 12.9, 13.8, 123.7, 19.7, 93.0, 4.2, 107.4, 44.1, 46.8]


In [24]:

sub_set = [[['-'] + [x/number_divisions for x in filtered_initial_set[-1][1:]]] for i in range(number_divisions)]
sub_set

[[['-', 40.8, 12.9, 13.8, 123.7, 19.7, 93.0, 4.2, 107.4, 44.1, 46.8]],
 [['-', 40.8, 12.9, 13.8, 123.7, 19.7, 93.0, 4.2, 107.4, 44.1, 46.8]],
 [['-', 40.8, 12.9, 13.8, 123.7, 19.7, 93.0, 4.2, 107.4, 44.1, 46.8]],
 [['-', 40.8, 12.9, 13.8, 123.7, 19.7, 93.0, 4.2, 107.4, 44.1, 46.8]],
 [['-', 40.8, 12.9, 13.8, 123.7, 19.7, 93.0, 4.2, 107.4, 44.1, 46.8]],
 [['-', 40.8, 12.9, 13.8, 123.7, 19.7, 93.0, 4.2, 107.4, 44.1, 46.8]],
 [['-', 40.8, 12.9, 13.8, 123.7, 19.7, 93.0, 4.2, 107.4, 44.1, 46.8]],
 [['-', 40.8, 12.9, 13.8, 123.7, 19.7, 93.0, 4.2, 107.4, 44.1, 46.8]],
 [['-', 40.8, 12.9, 13.8, 123.7, 19.7, 93.0, 4.2, 107.4, 44.1, 46.8]],
 [['-', 40.8, 12.9, 13.8, 123.7, 19.7, 93.0, 4.2, 107.4, 44.1, 46.8]]]

## 
ITERATIVE STRATIFICATION

- while there're still sentences in the initial_set
    - update number of each label in the initial_set
    - select label in initial_set that has fewest (at least 1) example
    - then for each sentences that contains this label,
        - find the subset that desires it more and add it to it
        - update the desired number of each label for the subset

In [25]:
# initial_set_l = [x for x in initial_set[:-1] if x[2] > 0]
# len(initial_set_l)
# 12168 - 2734
# for sentence in initial_set_l:
#     initial_set.pop(initial_set.index(sentence))
    

In [26]:
# len(initial_set)
# initial_set[-1]
# initial_set[-1] = ['-'] + [ sum(x) for x in zip(*(row[1:] for row in initial_set[:-1])) ]
# initial_set[-1]

In [27]:
while len(filtered_initial_set) > 1:
    print(f"\nTamanho atual de filtered_initial_set: {len(filtered_initial_set)}")
    contador = 0

    try:
        # Identificar o índice da label com menor quantidade
        min_lbl_idx = filtered_initial_set[-1].index(min((i for i in filtered_initial_set[-1][1:] if i > 0)))
        print(f"Label com menor quantidade: {min_lbl_idx} (quantidade: {filtered_initial_set[-1][min_lbl_idx]})")

        # Filtrar as sentenças relevantes
        initial_set_l = [x for x in filtered_initial_set[:-1] if x[min_lbl_idx] > 0]

        # Ordenar as sentenças pela quantidade da palavra desejada (min_lbl_idx), em ordem decrescente
        initial_set_l.sort(key=lambda x: x[min_lbl_idx], reverse=True)

        print(f"\nSentenças ordenadas pela quantidade da palavra desejada (label {min_lbl_idx}):")
        for idx, item in enumerate(initial_set_l):
            print(f"  {idx + 1}: {item}")

        # Inicializar o índice das partições
        partition_order = list(range(10))  # 10 partições

        for sentence in initial_set_l:
            partition_found = False

            # Ordenar as partições pela quantidade de desejo da min_lbl_idx, em ordem decrescente
            sorted_partitions = sorted(partition_order, key=lambda idx: sub_set[idx][0][min_lbl_idx], reverse=True)

            for partition_idx in sorted_partitions:
                current_partition_desires = sub_set[partition_idx][0][1:]
                potential_desires = [
                    current - count
                    for current, count in zip(current_partition_desires, sentence[1:])
                ]

                # Verificar se algum desejo se torna negativo
                if all(potential >= 0 for potential in potential_desires):
                    # Se a partição permitir a adição sem causar desejos negativos, adicione a sentença
                    sub_set[partition_idx].append(sentence)
                    filtered_initial_set.pop(filtered_initial_set.index(sentence))

                    # Atualizar o desejo de cada label na partição
                    sub_set[partition_idx][0][1:] = [
                        current - count for current, count in zip(sub_set[partition_idx][0][1:], sentence[1:])
                    ]

                    # Atualizar os valores em filtered_initial_set
                    filtered_initial_set[-1] = ['-'] + [sum(x) for x in zip(*(row[1:] for row in filtered_initial_set[:-1]))]

                    # Imprimir status atualizado
                    total_entities_in_partition = sum(
                        sum(doc[1:]) for doc in sub_set[partition_idx][1:]
                    )
                    print(f"\nSentença adicionada: {sentence}")
                    print(f"Partição {partition_idx} agora tem {total_entities_in_partition} entidades.")

                    # Imprimir o desejo de cada partição
                    print("\nDesejo de cada partição após adicionar a sentença:")
                    for i, partition in enumerate(sub_set):
                        print(f"Partição {i}: {partition[0][1:]}")

                    partition_found = True
                    break  # Sentença adicionada, pode sair do loop

            if not partition_found:
                # Caso a sentença não tenha sido alocada sem negativos, vamos escolher a partição com o menor número de entidades negativas
                best_negative_count = float('inf')  # Maior soma de negativos
                best_negative_sum = float('inf')   # Soma dos negativos
                best_partition_idx = None

                for partition_idx in sorted_partitions:
                    current_partition_desires = sub_set[partition_idx][0][1:]
                    potential_desires = [
                        current - count
                        for current, count in zip(current_partition_desires, sentence[1:])
                    ]

                    # Contar quantos desejos se tornam negativos
                    negative_count = sum(1 for desire in potential_desires if desire < 0)

                    # Calcular a soma dos valores negativos
                    negative_sum = sum(1 for desire in potential_desires if desire < 0)

                    # Se o número de negativos for menor, ou se for igual e a soma dos negativos for maior, escolher esta partição
                    if negative_count < best_negative_count or (negative_count == best_negative_count and negative_sum > best_negative_sum):
                        best_negative_count = negative_count
                        best_negative_sum = negative_sum
                        best_partition_idx = partition_idx

                if best_partition_idx is not None:
                    # Adicionar a sentença na partição com o menor impacto negativo
                    sub_set[best_partition_idx].append(sentence)
                    sub_set[best_partition_idx][0][1:] = [
                        current - count for current, count in zip(sub_set[best_partition_idx][0][1:], sentence[1:])
                    ]
                    filtered_initial_set.pop(filtered_initial_set.index(sentence))

                    # Atualizar os valores em filtered_initial_set
                    filtered_initial_set[-1] = ['-'] + [sum(x) for x in zip(*(row[1:] for row in filtered_initial_set[:-1]))]

                    # Imprimir status atualizado
                    total_entities_in_partition = sum(
                        sum(doc[1:]) for doc in sub_set[best_partition_idx][1:]
                    )
                    print(f"\nSentença adicionada (com negativos): {sentence}")
                    print(f"Partição {best_partition_idx} agora tem {total_entities_in_partition} entidades.")

                    # Imprimir o desejo de cada partição
                    print("\nDesejo de cada partição após adicionar a sentença:")
                    for i, partition in enumerate(sub_set):
                        print(f"Partição {i}: {partition[0][1:]}")

            contador += 1

        # Recalcular a label com menor quantidade após a alocação das sentenças
        min_lbl_idx = filtered_initial_set[-1].index(min((i for i in filtered_initial_set[-1][1:] if i > 0)))

    except ValueError:
        # Caso ValueError, processar as sentenças restantes sem basear-se na label
        initial_set_l = filtered_initial_set[:-1]

        print(f"\nConteúdo de initial_set_l (ValueError handler, {len(initial_set_l)} elementos):")
        for idx, item in enumerate(initial_set_l):
            print(f"  {idx + 1}: {item}")

        for sentence in initial_set_l:
            # Verificar os tamanhos das partições
            partition_sizes = [sum(sum(doc[1:]) for doc in partition[1:]) for partition in sub_set]
            partitions_below_limit = [i for i, size in enumerate(partition_sizes) if size < 1225]

            # Escolher a partição com menor valor total
            if partitions_below_limit:
                target_partition_idx = min(partitions_below_limit, key=lambda i: partition_sizes[i])
            else:
                target_partition_idx = partition_sizes.index(min(partition_sizes))

            # Adicionar a sentença na partição escolhida
            sub_set[target_partition_idx].append(sentence)
            filtered_initial_set.pop(filtered_initial_set.index(sentence))

            # Atualizar os valores em filtered_initial_set
            filtered_initial_set[-1] = ['-'] + [sum(x) for x in zip(*(row[1:] for row in filtered_initial_set[:-1]))]

            # Imprimir status atualizado
            total_entities_in_partition = partition_sizes[target_partition_idx] + sum(sentence[1:])
            print(f"\nSentença adicionada: {sentence}")
            print(f"Partição {target_partition_idx} agora tem {total_entities_in_partition} entidades.")

    print(f"\nEstado atualizado de filtered_initial_set: {filtered_initial_set[-1]}")
    print(f"Total de sentenças processadas nesta iteração: {contador}")



Tamanho atual de filtered_initial_set: 130
Label com menor quantidade: 7 (quantidade: 42)

Sentenças ordenadas pela quantidade da palavra desejada (label 7):
  1: [52, 0, 0, 24, 1, 0, 0, 8, 0, 9, 7]
  2: [82, 0, 4, 0, 12, 0, 6, 5, 0, 2, 4]
  3: [124, 7, 0, 0, 12, 5, 40, 4, 14, 3, 5]
  4: [26, 17, 3, 2, 3, 0, 9, 3, 0, 3, 2]
  5: [40, 1, 3, 0, 0, 0, 11, 3, 5, 4, 1]
  6: [47, 4, 0, 0, 6, 1, 7, 2, 11, 4, 0]
  7: [67, 3, 0, 4, 10, 0, 13, 2, 0, 7, 3]
  8: [103, 2, 0, 0, 30, 0, 0, 2, 10, 6, 0]
  9: [110, 7, 7, 0, 11, 2, 3, 2, 10, 1, 1]
  10: [5, 8, 0, 0, 2, 2, 5, 1, 1, 4, 2]
  11: [17, 0, 2, 0, 1, 4, 0, 1, 4, 2, 3]
  12: [28, 10, 0, 0, 7, 17, 1, 1, 3, 13, 0]
  13: [57, 1, 0, 0, 26, 0, 8, 1, 1, 7, 3]
  14: [60, 0, 0, 0, 1, 1, 4, 1, 1, 1, 1]
  15: [79, 0, 0, 0, 9, 1, 0, 1, 23, 3, 2]
  16: [80, 0, 1, 1, 12, 2, 3, 1, 22, 3, 3]
  17: [85, 6, 2, 0, 6, 9, 8, 1, 8, 17, 0]
  18: [109, 1, 0, 0, 3, 0, 1, 1, 11, 1, 0]
  19: [119, 0, 2, 0, 6, 0, 21, 1, 2, 8, 0]
  20: [125, 6, 10, 0, 3, 0, 9, 1, 0, 3, 1]


In [28]:
filtered_initial_set

[['-']]

In [29]:
for sub in sub_set:
    print('tam: {} - lbs:{}'.format(len(sub[1:]), sub[0]))

tam: 7 - lbs:['-', 13.799999999999997, 10.9, -47.2, 6.700000000000003, 8.7, -22.0, -3.8, 27.400000000000006, 12.100000000000001, 4.799999999999997]
tam: 14 - lbs:['-', 5.799999999999997, 0.9000000000000004, 9.8, -51.3, 0.6999999999999993, 13.0, -0.7999999999999998, 0.4000000000000057, 0.10000000000000142, -1.2000000000000028]
tam: 12 - lbs:['-', 0.7999999999999972, -0.09999999999999964, 9.8, 22.700000000000003, 0.6999999999999993, -35.0, 0.20000000000000018, 42.400000000000006, 0.10000000000000142, 0.7999999999999972]
tam: 12 - lbs:['-', 4.799999999999997, -16.1, 10.8, 8.700000000000003, 9.7, 34.0, 0.20000000000000018, -2.5999999999999943, 3.1000000000000014, -25.200000000000003]
tam: 14 - lbs:['-', -9.200000000000003, 0.9000000000000004, 0.8000000000000007, 5.700000000000003, -9.3, -15.0, 0.20000000000000018, 16.400000000000006, 0.10000000000000142, 3.799999999999997]
tam: 13 - lbs:['-', -11.200000000000003, 0.9000000000000004, 6.800000000000001, 13.700000000000003, 0.6999999999999993

In [30]:
for sub in sub_set:
    
    print (sub)
    

[['-', 13.799999999999997, 10.9, -47.2, 6.700000000000003, 8.7, -22.0, -3.8, 27.400000000000006, 12.100000000000001, 4.799999999999997], [52, 0, 0, 24, 1, 0, 0, 8, 0, 9, 7], [59, 0, 1, 14, 5, 0, 7, 0, 0, 3, 10], [120, 3, 1, 23, 11, 0, 5, 0, 1, 1, 4], [41, 15, 0, 0, 53, 5, 9, 0, 19, 11, 15], [21, 3, 0, 0, 26, 4, 62, 0, 34, 2, 2], [77, 6, 0, 0, 11, 2, 2, 0, 14, 2, 2], [75, 0, 0, 0, 10, 0, 30, 0, 12, 4, 2]]
[['-', 5.799999999999997, 0.9000000000000004, 9.8, -51.3, 0.6999999999999993, 13.0, -0.7999999999999998, 0.4000000000000057, 0.10000000000000142, -1.2000000000000028], [82, 0, 4, 0, 12, 0, 6, 5, 0, 2, 4], [101, 7, 2, 0, 54, 1, 2, 0, 33, 7, 13], [107, 0, 2, 0, 2, 0, 3, 0, 6, 4, 0], [18, 7, 1, 0, 9, 1, 5, 0, 1, 1, 0], [23, 1, 1, 0, 5, 1, 17, 0, 1, 9, 2], [27, 0, 1, 0, 3, 3, 1, 0, 4, 5, 2], [39, 3, 1, 0, 6, 6, 5, 0, 8, 5, 1], [105, 0, 0, 2, 6, 0, 2, 0, 10, 0, 2], [126, 5, 0, 2, 7, 0, 6, 0, 3, 2, 10], [13, 0, 0, 0, 5, 5, 9, 0, 15, 1, 4], [42, 1, 0, 0, 3, 2, 19, 0, 16, 1, 6], [114, 1, 0, 0,

In [31]:
len(sub_set[-1])

18

In [32]:
import os

# Iterando sobre cada subconjunto em sub_set
for idx, s_set in enumerate(sub_set):
    # Construindo o caminho do arquivo para o subconjunto, com extensão .txt
    file_path = f"../Base de Dados/HAREM/divisions/first/iterative/division_{idx}.txt"
    
    # Verificar se o diretório existe, caso contrário, criá-lo
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Diretório criado: {directory}")
    
    # Abrindo o arquivo para o subconjunto
    with open(file_path, 'w+', encoding='utf-8') as fp:
        # Iterando sobre cada documento no subconjunto, excluindo a linha de contagens (assumindo que está na posição 0)
        for set_doc in s_set[1:]:
            # Obter o nome do documento usando o índice do documento
            doc_name = list(data_por_sentenca_por_doc.keys())[set_doc[0]]
            doc_sentences = data_por_sentenca_por_doc[doc_name]

            # Escrever cada sentença do documento no arquivo
            for sentence in doc_sentences:
                for tk_class in sentence:
                    # Gravar cada elemento da sentença, com separador adequado
                    fp.write(config['col_sep'].join(map(str, tk_class)) + '\n')
                # Quebra de linha entre sentenças
                fp.write('\n')
    
    print(f"Subconjunto {idx} salvo com sucesso em {file_path}")



Subconjunto 0 salvo com sucesso em ../Base de Dados/HAREM/divisions/first/iterative/division_0.txt
Subconjunto 1 salvo com sucesso em ../Base de Dados/HAREM/divisions/first/iterative/division_1.txt
Subconjunto 2 salvo com sucesso em ../Base de Dados/HAREM/divisions/first/iterative/division_2.txt
Subconjunto 3 salvo com sucesso em ../Base de Dados/HAREM/divisions/first/iterative/division_3.txt
Subconjunto 4 salvo com sucesso em ../Base de Dados/HAREM/divisions/first/iterative/division_4.txt
Subconjunto 5 salvo com sucesso em ../Base de Dados/HAREM/divisions/first/iterative/division_5.txt
Subconjunto 6 salvo com sucesso em ../Base de Dados/HAREM/divisions/first/iterative/division_6.txt
Subconjunto 7 salvo com sucesso em ../Base de Dados/HAREM/divisions/first/iterative/division_7.txt
Subconjunto 8 salvo com sucesso em ../Base de Dados/HAREM/divisions/first/iterative/division_8.txt
Subconjunto 9 salvo com sucesso em ../Base de Dados/HAREM/divisions/first/iterative/division_9.txt


In [34]:


import os
import re
from collections import Counter
import matplotlib.pyplot as plt

# Diretório onde as partições estão salvas
base_dir = "../Base de Dados/HAREM/divisions/first/iterative/"

number_divisions = 10

# Inicializar lista para as classes teóricas
unique_lbl_list = []

# Lista de abreviações
abbreviations = r"\b(?:Sr|S|Sra|Dr|Prof|Eng|Mme|Mestre|Vº|Vª)\.$"  # Abreviações que não devem terminar uma sentença
pattern = r"[.!?](\s|$)"  # Verifica ponto, exclamação ou interrogação seguidos de espaço ou fim da linha

# Contadores gerais
total_counts = Counter()
total_sentences_por_div = {}
total_tokens_por_div = {}

for file_name in sorted(os.listdir(base_dir)):
    if file_name.startswith("division_"):
        partition_path = os.path.join(base_dir, file_name)
        with open(partition_path, 'r', encoding='utf-8') as file:
            sentence_count = 0
            token_count = 0
            for line in file:
                line = line.strip()
                if line:
                    token_data = line.split()
                    label = token_data[-1]
                    token_count += 1  # Contando tokens
                    if label.startswith("B-"):  # Considerar apenas os rótulos com prefixo 'B-'
                        simplified_label = label.split("-")[-1]
                        unique_lbl_list.append(simplified_label)
                        total_counts[simplified_label] += 1
                    
                    # Verificar se a linha termina com ponto, exclamação ou interrogação
                    # e garantir que não seja uma abreviação
                    if re.search(pattern, line) and not re.search(abbreviations, line):
                        sentence_count += 1
            
            total_sentences_por_div[file_name] = sentence_count
            total_tokens_por_div[file_name] = token_count

# Obter classes teóricas únicas e ordenadas
classes_teoricas = sorted(set(unique_lbl_list))

# Calcular os valores teóricos dividindo as contagens totais por número de divisões
valores_teoricos = [total_counts[cls] / number_divisions for cls in classes_teoricas]

# Inicializando dicionários para armazenar contagens por partição
qtd_classes_por_div = {}
total_entidades_por_div = {}

# Iterar novamente pelas partições para calcular os valores reais
for file_name in sorted(os.listdir(base_dir)):
    if file_name.startswith("division_"):
        partition_path = os.path.join(base_dir, file_name)

        # Inicializar contador para esta partição
        labels_count = Counter()

        with open(partition_path, 'r', encoding='utf-8') as file:
            for line in file:
                line = line.strip()
                if line:
                    token_data = line.split()
                    label = token_data[-1]
                    if label.startswith("B-"):  # Considerar apenas os rótulos com prefixo 'B-'
                        simplified_label = label.split("-")[-1]
                        labels_count[simplified_label] += 1

        # Armazenar as contagens para a partição atual
        qtd_classes_por_div[file_name] = labels_count
        total_entidades_por_div[file_name] = sum(labels_count.values())

# Calculando o total geral de todas as partições
total_entidades_geral = sum(total_entidades_por_div.values())
total_sentences_geral = sum(total_sentences_por_div.values())
total_tokens_geral = sum(total_tokens_por_div.values())

# Exibindo os totais por partição e o total geral
print("Totais de entidades, sentenças e tokens por partição:")
for div_name in total_entidades_por_div:
    print(f"  {div_name}: {total_entidades_por_div[div_name]} entidades, "
          f"{total_sentences_por_div[div_name]} sentenças, "
          f"{total_tokens_por_div[div_name]} tokens")
print(f"\nTotal geral de entidades: {total_entidades_geral}")
print(f"Total geral de sentenças: {total_sentences_geral}")
print(f"Total geral de tokens: {total_tokens_geral}\n")

# Gerando os gráficos para cada partição
for div_name, labels_count in qtd_classes_por_div.items():
    # Ordenar as contagens conforme as classes teóricas
    qtd = [labels_count.get(cls, 0) for cls in classes_teoricas]

    plt.figure(figsize=(10, 6))
    bars_real = plt.barh(classes_teoricas, qtd, color="pink", label="Contagem Real")
    bars_teorico = plt.barh(classes_teoricas, valores_teoricos, color="lightblue", alpha=0.5, label="Valor Teórico")

    # Adicionando os valores nas barras
    for i, count in enumerate(qtd):
        plt.text(count, i, str(count), va="center_baseline", ha="right", fontsize=10)

    for i, teorico in enumerate(valores_teoricos):
        if teorico > 0:
            plt.text(teorico, i, f"{teorico:.1f}", va="center_baseline", ha="right", fontsize=10, color="blue")

    plt.title(f"Entities per Class in Partition: {div_name}", fontsize=15)
    plt.xlabel("Number of Entities", fontweight="bold", fontsize=12)
    plt.ylabel("Classes", fontweight="bold", fontsize=12)

    plt.legend()

    # Salvando o gráfico
    output_path = os.path.join(base_dir, f"entities_per_class_{div_name}.png")
    plt.savefig(output_path, dpi=300, format="png", bbox_inches="tight")
    print(f"Gráfico salvo: {output_path}")

print("Todos os gráficos foram gerados e salvos com sucesso!")


ModuleNotFoundError: No module named 'matplotlib'

In [35]:
jupyter kernelspec list



SyntaxError: invalid syntax (381907592.py, line 1)